In [3]:
import pymongo
from pymongo import MongoClient
import csv
import os
from petl import fromcsv, look, cut, tocsv 
import numpy as np
from StringIO import StringIO
import sklearn
from sklearn import linear_model
import plotly 
import plotly.plotly as py
import re
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from geopy.distance import vincenty
from scipy.stats import norm
from numpy import linspace
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from pylab import plot,show,hist,figure,title
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from scipy.stats import norm
import scipy.stats
from matplotlib.legend_handler import HandlerLine2D
import math
import matplotlib.mlab as mlab
%matplotlib inline

In [4]:
#load and merge the tables scrapted from public datatbase

def p2f(x):
    return float(x.strip('%'))/100

county2labor = pd.read_csv('CA_county_employment.csv').as_matrix()
for i in range(1,county2labor.shape[0]):
    county2labor[i,5] = p2f(county2labor[i,5])
county2LF=county2labor[:,[0,2,5]]
for i in range(1,county2LF.shape[0]):
    county2LF[i,1]=int(county2LF[i,1])
county2LF[0,2] ='unemploy_rate'

propTax = pd.read_csv('CA_propTax.csv').as_matrix()
propTax[22,7] = '0'
propTax[58,7] = '0'

for i in range(1,propTax.shape[0]):
    propTax[i,7] = p2f(propTax[i,7])
propTax[22,7] = np.median(propTax[:,7])
propTax[58,7] = np.median(propTax[:,7])

propTax[0,0]='County'
propTax = propTax[:,[0,7]]
propTax[0,1]='midTax'

zip2county = pd.read_csv('CA_zip2county.csv').as_matrix()
zip2county = zip2county[:,[0,2]]
zip2county[0,0]='zipcd'
zip2county[0,1]='County'

for i in range(1,zip2county.shape[0]):
    zip2county[i,0]=int(zip2county[i,0])


df1=pd.DataFrame(county2LF, columns=['County', 'Labor Force', 'unemploy_rate'])
df2=pd.DataFrame(propTax, columns=['County', 'midTax'])
df3=pd.DataFrame(zip2county, columns=['zipcd', 'County'])

['zipcd', 'County','Labor Force', 'unemploy_rate', 'midTax']

df_merge=df1.merge(df2,on='County')
df_merge_merge=df3.merge(df_merge,on='County')
df_zip2countyInfo = df_merge_merge.ix[1:]
#df_zip2countyInfo =['County', 'Labor Force', 'unemploy_rate','midTax','zipcd']

In [5]:
# load the previous transaction data 

%matplotlib inline
#%qtconsole
data = pd.read_csv('soldPend_home_all.csv').as_matrix()

low_memory=False
mylist=[]
u, indices =np.unique(data[:,3], return_index=True)
data_uni = data[indices,:]
data=data_uni
                            

idx=[0,1,5,7,18]
data[:,idx]=data[:,idx]/1000 #convert price unit to $k
data[:,11]=2016-data[:,11] #convert year_built to age

idx_forFlip=np.where(data[:,16]==True)
data[:,16]=1
data[idx_forFlip,16]=2

idx_takedown=np.where(data[:,6]==True)
data[:,6]=1
data[idx_takedown,6]=2


#Handeling the non-exist and mis-input zipcode. Currently just not using those data. 
#It can be modified in the future to get zipcode from latitude and longitude
# if those are available.

noZip = pd.isnull(data[:,2])
idx_noZip=np.where(noZip==True)
data = np.delete(data,idx_noZip,0)

for i in range(0,data.shape[0]):
    if (type(data[i,2]) is str):
        #print data[i,2]
        data[i,2] = int(data[i,2][:5])
        #print data[i,2]
idx_misZip=np.where(data[:,2]<90000)   
data = np.delete(data,idx_misZip,0)


idx_soldyearFilter=np.where(data[:,13]>10000)
data=np.delete(data,idx_soldyearFilter,0)
idx_lotFilter=np.where(data[:,12]>2*pow(10,11))
data=np.delete(data,idx_lotFilter,0)
idx_sqftFilter=np.where(data[:,17]>400000)
data=np.delete(data,idx_sqftFilter,0)
idx_misSqft=np.where(data[:,17]<20)
data=np.delete(data,idx_misSqft,0)
                 
idx_priceFilter=np.where(data[:,18]>200000)
data=np.delete(data,idx_priceFilter,0)

#handling the unreasonable and irrelevant year_built
idx_yearFilter=np.where(data[:,11]<0)
data[idx_yearFilter,11]=np.median(data[:,11])
idx_yearFilter=np.where(data[:,11]>500)
data[idx_yearFilter,11]=np.median(data[:,11])

data_forComp=data

col_null = pd.isnull(data[:,1])
idx_prepriceExist=np.where(col_null==True)
data=np.delete(data,idx_prepriceExist,0)

/Users/Cindy/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
#imputing #bed and #bath
numBath_mis=np.where(data[:,4]>100) #remove an outlier in the numBath
data=np.delete(data,numBath_mis,0)

noBath = pd.isnull(data[:,4])
idx_noBath=np.where(noBath==True)
data[idx_noBath,4]=np.median(data[:,4])

idx_0Bath = np.where(data[:,4]==0)
data[idx_0Bath,4]=np.median(data[:,4])

noBed = pd.isnull(data[:,10])
idx_noBed=np.where(noBed==True)
data[idx_noBed,9]=np.median(data[:,10])

idx_0Bed = np.where(data[:,10]==0)
data[idx_0Bed,9]=np.median(data[:,10])

#drop the rows where there are still features missing
for i in range(0,19):
    col_null = pd.isnull((data[:,i]))
    idx=np.where(col_null==True)
    data=np.delete(data,idx,0)

In [8]:
# merge the previous transaction data with transaction data

df=pd.DataFrame(data, columns=['pre_sqft_price','pre_price','zipcd','propID','numBath','sqft_price','takedown',\
                               'cur_lis_price','lat','long','numBed','year_built','lot_sqft','numDays','year',\
                               'month','forFlip','sqft','price'])

df_statbyZip = df['price'].astype(float).groupby(df['zipcd']).agg([np.median,np.mean,np.std,np.max,np.min,len])

data_statbyZip = df_statbyZip.as_matrix()
b=(np.unique(data[:,2])).reshape((df_statbyZip.shape[0],1))
data_statbyZip=np.append(data_statbyZip,b,1)

df_statbyZip=pd.DataFrame(data_statbyZip, columns=['median','mean','std','max','min','count','zipcd'])
df_additional = df_statbyZip.merge(df_zip2countyInfo,on='zipcd')
df_all=df.merge(df_additional,on='zipcd')

data_all=df_all.as_matrix()
data_all[:,[18,28]]=data_all[:,[28,18]]
data_all[:,[0,3]]=data_all[:,[3,0]]
data_all[:,[1,2]]=data_all[:,[2,1]]
data_all[:,[8,9]]=data_all[:,[9,8]]
#remove houses that has most likely mis input of sqft and which that has unreasonalbe price per sqft

var_noNeed=[3,5,7,20,21,22,23,25]
data_model=np.delete(data_all,var_noNeed,1)  #delete the variable that are not needed



In [ ]:
# to disply the columes in data_all
fields_data_all=['propID','zipcd','pre_price','pre_sqft_price','numBath','sqft_price','takedown','cur_lis_price',\
             'lat','long','numBed','year_built','lot_sqft','numDays','year','month',\
             'forFlip','sqft','midTax','median','mean','std','max','min','len','County','Labor Force',\
        'unemploy_rate','price']
for index in np.arange(len(fields_data_all)):
    print str(index) + "," + fields_data_all[index]

In [ ]:
# to disply the columes in data_model
fields_data_model=['propID','zipcd','pre_price','numBath','takedown',\
             'lat','long','numBed','year_built','lot_sqft','numDays','year','month',\
             'forFlip','sqft','midTax','median','len','Labor Force','unemploy_rate','price']
for index in np.arange(len(fields_data_model)):
    print str(index) + "," + fields_data_model[index]

In [ ]:
def pctErr_top10comp(testID):
    
    Y_train_compa = np.delete(data_model[:,20],testID,0)
    
    feature1_compa=np.delete(data_model[:,:20],testID,0)
    loc_test = data_model[testID,:20]
    Y_test = data_model[testID,20]
    idx_numDaysFilter_compa = np.where(feature1_compa[:,10]>(loc_test[10]+180))
    idx_numDaysFilter_compa = np.asarray(idx_numDaysFilter_compa)
    feature1_compa=np.delete(feature1_compa,idx_numDaysFilter_compa,0)
    Y_train_compa=np.delete(Y_train_compa,idx_numDaysFilter_compa,0)

    idx_numDaysFilter2_compa = np.where(feature1_compa[:,10] < loc_test[10])
    idx_numDaysFilter2_compa = np.asarray(idx_numDaysFilter2_compa)
    feature1_compa=np.delete(feature1_compa,idx_numDaysFilter2_compa,0)
    Y_train_compa=np.delete(Y_train_compa,idx_numDaysFilter2_compa,0)   
    
    idx_flipFilter_compa = np.where(feature1_compa[:,13] != loc_test[13])
    idx_flipFilter_compa = np.asarray(idx_flipFilter_compa)
    feature1_compa=np.delete(feature1_compa,idx_flipFilter_compa,0)
    Y_train_compa=np.delete(Y_train_compa,idx_flipFilter_compa,0)

    idx_tdFilter_compa = np.where(feature1_compa[:,4] != loc_test[4])
    idx_tdFilter_compa = np.asarray(idx_tdFilter_compa)
    feature1_compa=np.delete(feature1_compa,idx_tdFilter_compa,0)
    Y_train_compa=np.delete(Y_train_compa,idx_tdFilter_compa,0)    
    feature_test_compa =np.append(loc_test, 0)    
    Y_train_compa_backup = Y_train_compa
    feature1_compa_backup=feature1_compa
    
    def price_sqft_filter(price_match,sqft_match,Y_train_compa_backup,feature1_compa_backup,loc_test):
        idx_priceFilter_compa = np.where(np.absolute(np.divide(Y_train_compa_backup-loc_test[2],loc_test[2]))>price_match)
        idx_priceFilter_compa = np.asarray(idx_priceFilter_compa)
        feature1_compa=np.delete(feature1_compa_backup,idx_priceFilter_compa,0)
        Y_train_compa=np.delete(Y_train_compa_backup,idx_priceFilter_compa,0)

        idx_sqftFilter_compa = np.where(np.abs(np.divide(feature1_compa[:,14]-loc_test[14],loc_test[14]))>sqft_match)
        idx_sqftFilter_compa = np.asarray(idx_sqftFilter_compa)
        feature1_compa=np.delete(feature1_compa,idx_sqftFilter_compa,0)
        Y_train_compa=np.delete(Y_train_compa,idx_sqftFilter_compa,0)
        return (feature1_compa, Y_train_compa)
    
    price_match = 0.1
    sqft_match = 0.2
    feature1_compa, Y_train_compa = price_sqft_filter(price_match,sqft_match,Y_train_compa_backup,feature1_compa_backup,loc_test)
    
    top_compa= 30   
    if feature1_compa.shape[0]<top_compa:
        if feature1_compa.shape[0]>10:
            top_compa = feature1_compa.shape[0]
        else:
            feature1_compa, Y_train_compa = price_sqft_filter(price_match+0.05,sqft_match+0.1,Y_train_compa_backup,feature1_compa_backup,loc_test)
            if feature1_compa.shape[0]<10:                
                feature1_compa, Y_train_compa = price_sqft_filter(price_match+0.1,sqft_match+0.2,Y_train_compa_backup,feature1_compa_backup,loc_test)              
                if feature1_compa.shape[0]<10:
                    feature1_compa, Y_train_compa = price_sqft_filter(price_match+0.3,sqft_match+0.5,Y_train_compa_backup,feature1_compa_backup,loc_test)              
                    if feature1_compa.shape[0]==0:
                        feature1_compa = loc_test.reshape((1,loc_test.shape[0]))
                        Y_train_compa = np.asarray([1]) 
            if feature1_compa.shape[0]<top_compa:
                top_compa = feature1_compa.shape[0]
    dist_compa=[]
    for i in range(0,feature1_compa.shape[0]):
            dist_compa.append(vincenty(loc_test[5:7], feature1_compa[i,5:7]).miles)
 
    dist_compa=np.asarray(dist_compa)
    dist_compa_sorted = np.sort(dist_compa)
        
    dist_select_compa= dist_compa_sorted[0:top_compa]
    loc_match_id_compa=[np.where(dist_compa == dist_select_compa[x])[0] for x in range(0,top_compa)]
    loc_match_id_compa = np.concatenate(loc_match_id_compa).ravel()
    loc_match_id_compa = np.unique(loc_match_id_compa)
    dist_selected_compa = dist_compa_sorted[0:top_compa]
    dist_feature_compa = dist_compa[loc_match_id_compa]
    Y_train_compa = Y_train_compa[loc_match_id_compa]


    feature1_compa = feature1_compa[loc_match_id_compa,:]
    feature_compa=np.vstack((feature1_compa.T, dist_feature_compa.T))
    feature_compa=feature_compa.T

    feature_compa = np.vstack(feature_compa[:, 3:]).astype(np.float)
    feature_test_compa = np.vstack(feature_test_compa[3:]).astype(np.float)

    scaler_compa = preprocessing.StandardScaler().fit(feature_compa)
    X_scaled_compa = scaler_compa.transform(feature_compa)  
    X_scaled_normed_compa = preprocessing.normalize(X_scaled_compa, norm='l2')

    X_train_compa = X_scaled_normed_compa

    X_test_scaled_compa = scaler_compa.transform(feature_test_compa.reshape(1, -1))  
    X_test_scaled_normed_compa= preprocessing.normalize(X_test_scaled_compa.reshape(1, -1) , norm='l2')
    X_test_compa = X_test_scaled_normed_compa

    print str(X_train_compa.shape[0]) + (' houses are used for comparable house search')

    import math
    from itertools import izip

    def dot_product(v1, v2):
        return sum(map(lambda x: x[0] * x[1], izip(v1, v2)))

    def cosine_measure(v1, v2):
        prod = dot_product(v1, v2)
        len1 = math.sqrt(dot_product(v1, v1))
        len2 = math.sqrt(dot_product(v2, v2))
        return prod / (len1 * len2)

    similarity = []
    for i in range(0,X_train_compa.shape[0]):
        similarity.append(cosine_measure(X_test_compa.reshape((-1,1)), X_train_compa[i,:].reshape((-1,1))) )



    top_simi = 10
    if len(similarity)<30:
        top_simi = len(similarity)
    simi_sorted = sorted(similarity, reverse=True)
    simi_select = simi_sorted[0:top_simi]
    simi_match_id =[np.where(similarity == simi_select[x])[0] for x in range(0,top_simi)]
    simi_match_id = np.concatenate(simi_match_id).ravel()
    simi_match_id = np.unique(simi_match_id)  
    sold_price = Y_train_compa[simi_match_id]
    simi = np.asarray(similarity)[simi_match_id]
    simi=simi.reshape((-1,1))   
    weight = np.asarray(simi/np.sum(simi)).reshape((simi.shape[0],))
    price_est_weighted=np.sum(np.multiply(sold_price,weight))
    price_est_median=np.median(sold_price)
    err_median = price_est_median-Y_test
    err_weighted = price_est_weighted-Y_test
    pctErr_median = np.divide(err_median,Y_test)
    pctErr_weighted = np.divide(err_weighted,Y_test)
    pctErr_median=pctErr_median
    return (pctErr_median, pctErr_weighted,Y_test)   

pctErrM = []
pctErrW = []
Y=[]
num_test=0
group=0
for i in range(0,280000,1000):
    pctErr_median, pctErr_weighted,Y_test = pctErr_top10comp(i)
    pctErrM.append(pctErr_median)
    pctErrW.append(pctErr_weighted)
    Y.append(Y_test)
    num_test=num_test+1
    
    filename='pctErr'+str(group)
    if num_test == 20:
        scipy.io.savemat('pctErr'+str(group)+'_top5000.mat', {'pctErrMedian':pctErrM,'pctErrWeighted':pctErrW,'Y_test':Y})
        group=group+1
        num_test=num_test-20